In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from zipfile import ZipFile
import os
import cv2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import math
import random

In [2]:
os.chdir('.\\Desktop\\tensorflow\\Cactus_identification')
os.listdir(os.getcwd())

['aerial-cactus-identification.zip', 'data']

In [3]:
os.mkdir('data')
os.mkdir('data\\train')

In [4]:
with ZipFile('aerial-cactus-identification.zip', 'r') as file:
    
    file.extractall('data')

In [5]:
os.listdir('data')

['sample_submission.csv', 'test.zip', 'train', 'train.csv', 'train.zip']

In [6]:
with ZipFile('data\\train.zip', 'r') as file:
    
    file.extractall('data\\train')

In [7]:
os.mkdir('data\\test')
with ZipFile('data\\test.zip', 'r') as file:
    
    file.extractall('data\\test')

In [3]:
train_info = pd.read_csv('data\\train.csv')
train_info

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1
...,...,...
17495,ffede47a74e47a5930f81c0b6896479e.jpg,0
17496,ffef6382a50d23251d4bc05519c91037.jpg,1
17497,fff059ecc91b30be5745e8b81111dc7b.jpg,1
17498,fff43acb3b7a23edcc4ae937be2b7522.jpg,0


In [4]:
train_id = train_info['id'].values
train_label = train_info['has_cactus'].values

def load_data(source, data_id, size, train_label):
    
    data = []
    n = 0
    
    for file in data_id:
        
        img = os.path.join(source, file)
        img_raw = cv2.imread(img, cv2.COLOR_BGR2RGB)
        img_array = cv2.resize(img_raw, (size, size))
        data.append([img_array, train_label[n]])
        n += 1
        
    return data

In [5]:
img_data = load_data('data\\train\\train', train_id, 128, train_label)

In [6]:
img_data = np.array(img_data)
random.shuffle(img_data)

In [7]:
def seperate_img_label(data):
    
    x = []
    y = []
    
    for feature, label in data:
        x.append(feature)
        y.append(label)
    
    return x, y

In [8]:
x, y = seperate_img_label(img_data)

In [9]:
x, y = np.array(x), np.array(y)
x.shape

(17500, 128, 128, 3)

In [10]:
x = x / 255.

In [11]:
shuffle_idx = np.random.randint(0, len(img_data), size = (len(img_data)))
split_point = math.ceil(0.6*len(shuffle_idx))
train_x, train_y = x[0:split_point], y[0:split_point]
val_x, val_y = x[split_point:], y[split_point:]
print('Number of training samples: ', train_x.shape[0])
print('Number of validation samples: ', val_x.shape[0])

Number of training samples:  10500
Number of validation samples:  7000


In [12]:
train_x.shape

(10500, 128, 128, 3)

Another way (probably better way when using tensorflow) to preprocess the data is to use the keras.preprocessing.image.ImageDataGenerator.flow_from_dataframe API.

-One thing worth noticing is that flow_from_dataframe requires that the target column must be string formatted.

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [58]:
gen_train_df = train_info.copy()
gen_train_df['has_cactus'] = gen_train_df['has_cactus'].astype(str)

In [80]:
datagen = ImageDataGenerator(rescale = 1/255.)
shuffle_idx = np.random.randint(0, len(img_data), size = (len(img_data)))
split_point = math.ceil(0.6*len(shuffle_idx))

train_gen = datagen.flow_from_dataframe(dataframe = gen_train_df[:split_point], directory = 'data\\train\\train', 
                                        x_col = 'id',
                                       y_col = 'has_cactus', 
                                        class_mode = 'binary', batch_size = 50, shuffle = True,
                                 target_size = (128, 128))

val_gen = datagen.flow_from_dataframe(dataframe = gen_train_df[split_point:], directory = 'data\\train\\train', 
                                        x_col = 'id',
                                       y_col = 'has_cactus', 
                                        class_mode = 'binary', batch_size = 20, shuffle = True,
                                 target_size = (128, 128))

Found 10500 validated image filenames belonging to 2 classes.
Found 7000 validated image filenames belonging to 2 classes.


In [66]:
for batch in train_gen:
    print(batch[0].shape)
    break

(50, 128, 128, 3)


### Naive Inception model. 

The key idea is to use different type of convolutions models running in parallel, then concatenate all outputs together to generate prediction.

In [64]:
x = Input(shape = (train_x.shape[1], train_x.shape[2], train_x.shape[3]))

channel = Conv2D(64, 1, strides = 2, padding = 'same', activation = 'relu')(x)

channel_by_scan = Conv2D(32, 1, strides = 1, padding = 'same', activation = 'relu')(x)
channel_by_scan = Conv2D(64, (3, 3), strides = 2, padding = 'same', activation = 'relu')(channel_by_scan)
channel_by_scan = BatchNormalization()(channel_by_scan)

spatial_capture = AveragePooling2D(3, strides = 2, padding = 'same')(x)
spatial_capture = Conv2D(64, (3, 3), strides = 1, padding = 'same', activation = 'relu')(spatial_capture)
spatial_capture = BatchNormalization()(spatial_capture)

pattern_capture = Conv2D(32, 1, strides = 1, padding = 'same', activation = 'relu')(x)
pattern_capture = Conv2D(64, (3, 3), strides = 1, padding = 'same', activation = 'relu')(pattern_capture)
pattern_capture = Conv2D(64, (3, 3), strides = 2, padding = 'same', activation = 'relu')(pattern_capture)
pattern_capture = BatchNormalization()(pattern_capture)

concated = concatenate([channel, channel_by_scan, spatial_capture, pattern_capture], axis = -1)

flat = Flatten()(concated)

dense = Dense(128, activation = 'relu')(flat)
dense = Dropout(0.5)(dense)
output = Dense(1, activation = 'sigmoid')(dense)

In [65]:
model = Model(x, output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 128, 128, 32) 128         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 128, 128, 32) 128         input_3[0][0]                    
__________________________________________________________________________________________________
average_pooling2d_2 (AveragePoo (None, 64, 64, 3)    0           input_3[0][0]                    
____________________________________________________________________________________________

In [66]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['acc'])

In [67]:
history = model.fit(train_x, train_y, batch_size = 32, epochs = 80, validation_data = (val_x, val_y))

Train on 10500 samples, validate on 7000 samples
Epoch 1/80
10500/10500 [==============================] - 44s 4ms/sample - loss: 0.2819 - acc: 0.8963 - val_loss: 0.3318 - val_acc: 0.8491
Epoch 2/80
10500/10500 [==============================] - 42s 4ms/sample - loss: 0.1426 - acc: 0.9450 - val_loss: 0.2122 - val_acc: 0.9209
Epoch 3/80
10500/10500 [==============================] - 42s 4ms/sample - loss: 0.1051 - acc: 0.9623 - val_loss: 0.1105 - val_acc: 0.9650
Epoch 4/80
10500/10500 [==============================] - 42s 4ms/sample - loss: 0.0760 - acc: 0.9720 - val_loss: 0.1165 - val_acc: 0.9621
Epoch 5/80
10500/10500 [==============================] - 42s 4ms/sample - loss: 0.0660 - acc: 0.9768 - val_loss: 0.1042 - val_acc: 0.9664
Epoch 6/80
10500/10500 [==============================] - 42s 4ms/sample - loss: 0.0591 - acc: 0.9795 - val_loss: 0.1173 - val_acc: 0.9651
Epoch 7/80
10500/10500 [==============================] - 43s 4ms/sample - loss: 0.0501 - acc: 0.9833 - val_loss: 0.1

### Spatial recognization model.

Hopefully this model can capture the difference of distribution between a cactus infected picture and a cactus free picture.
Without using the pooling layer, this model, inspired by DCGAN model discriminator idea (learn the difference between distributions of different classes), should be able to learn the distribution's differences between a cactus-infected picture and a cactus-free picture, although the assumption that there is a difference between two classes, is not verified. 

In [91]:
spatial_dist_model = Sequential()

spatial_dist_model.add(Conv2D(64, (1, 1), padding = 'same', input_shape = (train_x.shape[1], train_x.shape[2], train_x.shape[3]), activation = 'relu'))
spatial_dist_model.add(BatchNormalization()) 

spatial_dist_model.add(Conv2D(64, (3, 3), padding = 'same', input_shape = (train_x.shape[1], train_x.shape[2], train_x.shape[3]), activation = 'relu'))
spatial_dist_model.add(BatchNormalization())

spatial_dist_model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
spatial_dist_model.add(BatchNormalization())

spatial_dist_model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
spatial_dist_model.add(BatchNormalization())

spatial_dist_model.add(Flatten())
spatial_dist_model.add(Dropout(0.5))

spatial_dist_model.add(Dense(1, activation = 'sigmoid'))

In [93]:
spatial_dist_model.compile(optimizer = tf.keras.optimizers.Adam(3e-5), loss = 'binary_crossentropy', metrics = ['acc'])

In [94]:
spatial_dist_history = spatial_dist_model.fit(train_x, train_y, batch_size = 16, epochs = 50, validation_data = (val_x, val_y))

Train on 10500 samples, validate on 7000 samples
Epoch 1/50
10500/10500 [==============================] - 82s 8ms/sample - loss: 0.2641 - acc: 0.9544 - val_loss: 0.3040 - val_acc: 0.9477
Epoch 2/50
10500/10500 [==============================] - 80s 8ms/sample - loss: 0.0991 - acc: 0.9829 - val_loss: 0.3229 - val_acc: 0.9699
Epoch 3/50
10500/10500 [==============================] - 80s 8ms/sample - loss: 0.0612 - acc: 0.9897 - val_loss: 0.4599 - val_acc: 0.9634
Epoch 4/50
10500/10500 [==============================] - 80s 8ms/sample - loss: 0.0682 - acc: 0.9904 - val_loss: 0.4873 - val_acc: 0.9511
Epoch 5/50
10500/10500 [==============================] - 80s 8ms/sample - loss: 0.0686 - acc: 0.9904 - val_loss: 0.4613 - val_acc: 0.9719
Epoch 6/50
10500/10500 [==============================] - 81s 8ms/sample - loss: 0.0468 - acc: 0.9939 - val_loss: 0.4897 - val_acc: 0.9690
Epoch 7/50
10500/10500 [==============================] - 81s 8ms/sample - loss: 0.0448 - acc: 0.9944 - val_loss: 0.4

### Normal Conv2D model.

In [50]:
nor_model = Sequential()

nor_model.add(Conv2D(64, (3, 3), padding = 'same', input_shape = (train_x.shape[1], train_x.shape[2], train_x.shape[3]), activation = 'relu'))
nor_model.add(MaxPooling2D(2))
nor_model.add(BatchNormalization())

nor_model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
nor_model.add(MaxPooling2D(2))
nor_model.add(BatchNormalization())

nor_model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
nor_model.add(MaxPooling2D(2))
nor_model.add(BatchNormalization())

nor_model.add(GlobalMaxPooling2D())
nor_model.add(Dense(256))
nor_model.add(Dropout(0.3))
nor_model.add(Dense(1, activation = 'sigmoid'))

In [51]:
nor_model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['acc'])

In [60]:
nor_history = nor_model.fit(train_x, train_y, batch_size = 32, epochs = 30, validation_data = (val_x, val_y))

Train on 10500 samples, validate on 7000 samples
Epoch 1/30
10500/10500 [==============================] - 16s 2ms/sample - loss: 0.0031 - acc: 0.9993 - val_loss: 0.0322 - val_acc: 0.9911
Epoch 2/30
10500/10500 [==============================] - 16s 2ms/sample - loss: 9.4867e-04 - acc: 1.0000 - val_loss: 0.0297 - val_acc: 0.9916
Epoch 3/30
10500/10500 [==============================] - 16s 2ms/sample - loss: 7.9466e-04 - acc: 1.0000 - val_loss: 0.0299 - val_acc: 0.9914
Epoch 4/30
10500/10500 [==============================] - 16s 2ms/sample - loss: 0.0012 - acc: 0.9998 - val_loss: 0.0337 - val_acc: 0.9904
Epoch 5/30
10500/10500 [==============================] - 16s 2ms/sample - loss: 0.0012 - acc: 0.9998 - val_loss: 0.0295 - val_acc: 0.9919
Epoch 6/30
10500/10500 [==============================] - 16s 2ms/sample - loss: 9.8679e-04 - acc: 1.0000 - val_loss: 0.0310 - val_acc: 0.9910
Epoch 7/30
10500/10500 [==============================] - 16s 2ms/sample - loss: 0.0013 - acc: 0.9999 - v

### Seperable Conv 2D model

A Conv2D model may become very computational heavy when the input images and (or) the model is big. An alternative layer is the seperable convolutional layer which first conduct depthwise convolution then performs the usual pointwise convolution on the output of the depthwise convolution. 

The structure and codes for building a Separable conv 2d model is the same as building a simple Conv2D model, except that we are now calling the SeparableConv2D layer instead of the normal Conv2D.

Due to the mathmetical fundamentals behind it, the SeparableConv2D is computational easier compare to the Conv2D model and thus, we can afford to build a deeper model.

To speed up the training and further reduces computations, we can also add a 1 by 1 kernel layer that has less dimensions(number of filters) than that of the previous layer's output.

In [89]:
sep_model = Sequential()

sep_model.add(SeparableConv2D(32, (3, 3), activation = 'relu'))
sep_model.add(SeparableConv2D(64, (3, 3), activation = 'relu'))
sep_model.add(MaxPooling2D(2))
sep_model.add(BatchNormalization())

sep_model.add(SeparableConv2D(64, (3, 3), activation = 'relu'))
sep_model.add(SeparableConv2D(128, (3, 3), activation = 'relu'))
sep_model.add(MaxPooling2D(2))
sep_model.add(BatchNormalization())

sep_model.add(SeparableConv2D(64, (3, 3), activation = 'relu'))
sep_model.add(SeparableConv2D(128, (3, 3), activation = 'relu'))
sep_model.add(MaxPooling2D(2))
sep_model.add(BatchNormalization())

sep_model.add(SeparableConv2D(32, (1, 1), activation = 'relu'))

sep_model.add(Flatten())
sep_model.add(Dense(512, activation = 'relu'))
sep_model.add(Dropout(0.5))
sep_model.add(Dense(1, activation = 'sigmoid'))

In [90]:
sep_model.compile(optimizer = tf.keras.optimizers.Adam(lr = 5e-5), loss = 'binary_crossentropy', metrics = ['acc'])

In [91]:
sep_history = sep_model.fit(train_gen, steps_per_epoch = 210, epochs = 50, validation_data = val_gen, validation_steps = 350)

Epoch 1/50
210/210 [==============================] - 115s 546ms/step - loss: 0.3675 - acc: 0.8398 - val_loss: 0.6486 - val_acc: 0.7557
Epoch 2/50
210/210 [==============================] - 116s 550ms/step - loss: 0.1600 - acc: 0.9370 - val_loss: 0.4947 - val_acc: 0.7557
Epoch 3/50
210/210 [==============================] - 114s 542ms/step - loss: 0.1251 - acc: 0.9513 - val_loss: 0.2539 - val_acc: 0.9081
Epoch 4/50
210/210 [==============================] - 113s 539ms/step - loss: 0.0952 - acc: 0.9635 - val_loss: 0.1215 - val_acc: 0.9489
Epoch 5/50
210/210 [==============================] - 110s 525ms/step - loss: 0.0745 - acc: 0.9724 - val_loss: 0.1607 - val_acc: 0.9284
Epoch 6/50
210/210 [==============================] - 108s 516ms/step - loss: 0.0639 - acc: 0.9760 - val_loss: 0.0601 - val_acc: 0.9774
Epoch 7/50
210/210 [==============================] - 109s 519ms/step - loss: 0.0590 - acc: 0.9775 - val_loss: 0.0511 - val_acc: 0.9837
Epoch 8/50
210/210 [============================